In [1]:
import tensorflow as tf
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense, BatchNormalization
from keras.utils import to_categorical

In [15]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

## Import Training Data from QGIS

In [3]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Train.csv')
df.head()

,Class,Label,Band_1,Band_2,Band_3,Band_4
0,Urban Area,3,161,145,194,218
1,Urban Area,3,150,133,186,181
2,Urban Area,3,164,151,185,204
3,Urban Area,3,156,140,207,199
4,Urban Area,3,154,140,171,182


## Preprocessing

In [4]:
from sklearn.preprocessing import StandardScaler, LabelEncoder
ss = StandardScaler()

In [78]:
X = df.drop(['Class','Label'],axis=1)
X = ss.fit_transform(X)
y = df['Label']

y = LabelEncoder().fit_transform(y)
Y = pd.get_dummies(y).values

## Model Split

In [79]:
from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test = train_test_split(X,Y,test_size = 0.3, random_state = 42)

## Model building

In [80]:
model = Sequential()

model.add(Dense(32,activation='relu',input_shape = (4,)))
model.add(Dense(64,activation='relu'))
model.add(Dense(5,activation='softmax'))

In [81]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 32)                160       
                                                                 
 batch_normalization_1 (Bat  (None, 32)                128       
 chNormalization)                                                
                                                                 
 dense_4 (Dense)             (None, 64)                2112      
                                                                 
 dense_5 (Dense)             (None, 5)                 325       
                                                                 
Total params: 2725 (10.64 KB)
Trainable params: 2661 (10.39 KB)
Non-trainable params: 64 (256.00 Byte)
_________________________________________________________________


## Training

In [82]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [83]:
model.fit(X_train, y_train, epochs=50,validation_data=(X_test, y_test))

Epoch 1/50
5/5 [==============================] - 2s 59ms/step - loss: 1.6266 - accuracy: 0.1905 - val_loss: 1.5725 - val_accuracy: 0.2063
Epoch 2/50
5/5 [==============================] - 0s 16ms/step - loss: 1.3109 - accuracy: 0.4762 - val_loss: 1.4809 - val_accuracy: 0.3810
Epoch 3/50
5/5 [==============================] - 0s 16ms/step - loss: 1.0577 - accuracy: 0.6735 - val_loss: 1.3962 - val_accuracy: 0.7143
Epoch 4/50
5/5 [==============================] - 0s 13ms/step - loss: 0.8693 - accuracy: 0.8571 - val_loss: 1.3189 - val_accuracy: 0.7778
Epoch 5/50
5/5 [==============================] - 0s 16ms/step - loss: 0.7318 - accuracy: 0.8912 - val_loss: 1.2475 - val_accuracy: 0.8095
Epoch 6/50
5/5 [==============================] - 0s 11ms/step - loss: 0.5933 - accuracy: 0.9252 - val_loss: 1.1818 - val_accuracy: 0.8571
Epoch 7/50
5/5 [==============================] - 0s 11ms/step - loss: 0.5115 - accuracy: 0.9252 - val_loss: 1.1212 - val_accuracy: 0.8730
Epoch 8/50
5/5 [===========

## Prediction

In [84]:
from osgeo import gdal

In [85]:
ds = gdal.Open('/content/drive/MyDrive/Colab Notebooks/training.tif')
files = ds.ReadAsArray()
image = np.moveaxis(files, 0, -1)
image = np.reshape(image, [1151*1151, 4])

In [98]:
test = pd.DataFrame(image, dtype='int16', columns=['Band_1', 'Band_2', 'Band_3', 'Band_4'])
test.head()

,Band_1,Band_2,Band_3,Band_4
0,128,110,209,175
1,120,90,215,168
2,125,101,204,194
3,138,128,189,209
4,144,144,194,212


In [ ]:
y_pred_all = model.predict(test)